<a href="https://colab.research.google.com/github/ExCaLBBR/ExCaLBBR_Projects/blob/main/RaciallyBiasedDecisions/RaciallyBiasedDecisions-Intersectionality_SURG/code/RBD_Intersectionality_IAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reference: IAT data analysis script adapted from https://github.com/tomstafford/IAT**

In [ ]:
'''

Calculate IAT score from data generated by Robin's PsychoPy script

TQS Feb 2014

Based on

Greenwald, A. G., Nosek, B. A., & Banaji, M. R. (2003). Understanding and using the implicit association test: I. An improved scoring algorithm. Journal of personality and social psychology, 85(2), 1972-216.

'''

import pandas as pd #for dealing with csv import
import os # for joining paths and filenames sensibly
import numpy as np #for the population std
import glob # for finding csv data files
import platform # paths use different dividers on linux vs windows, so we need to test for this


# Import libraries

In [ ]:
import pandas as pd #for dealing with csv import
import os # for joining paths and filenames sensibly
import numpy as np #for the population std
import glob # for finding csv data files
import platform # paths use different dividers on linux vs windows, so we need to test for this

# Define util functions

In [ ]:
def adjustedmean(RTs,corrs,penalty):
    n=len(corrs) #trials
    n_errors=n-sum(corrs) #errors
    cor_RTs=np.array(corrs)*RTs #sum of correct RTs
    cor_mean=sum(cor_RTs)/sum(corrs)
    
    #mean with errors replaced with penalty value
    return cor_mean+(n_errors*penalty)/n



def exclude_slows(RTs,corrs,slowRT_limit):
    new_rt=[] #holding variables
    new_cr=[]
    for i in range(len(RTs)): #iterate over every item
        if RTs[i] < slowRT_limit: #if it isn't too fast, include RT and corr values
            new_rt.append(RTs[i])
            new_cr.append(corrs[i])
    
    return (new_rt, new_cr)



## -----------DEFINE PARAMETERS
print "setting parameters"

penalty=0.600 #penalty - in seconds - for incorrect responses
slowRT_limit=10 #threshold at which slow RTs are discarded
fastRT_limit=0.300 #threshold which defines responses which are "too fast"
fast_prop_limit=0.1 # threshold proportion of "too fast" responses which defines exclusion of ppt

#where we expect the data files to be
#search_string=os.path.join('..','IAT-1.3','data','*.csv') 
search_string=os.path.join('expt','data','*.csv') 
#we use os.path.join because the slashes go different ways on different operating systems
#ie path = '../IAT-1.1/data' #linux
#   path = '..\IAT-1.1\data' #windows

## -----------LOAD DATA
print "loading datafiles"

files = glob.glob(search_string) #list of data files in the named location

#now iterate through all data files and calculate the IAT
for filename in files:

    #import into python using pandas
    df = pd.read_csv(filename)
    
    ## -----------EXTRACT DATA
    print "extracting data & calculating IAT"
    
    #extract the stuff we're interested in (n.b i am indexing using the column names defined in the csv)
    #dropna() drops nans
    #tolist() converts from series to list
    corrs=df['key_resp_9.corr'].dropna().tolist()
    rts=df['key_resp_9.rt'].dropna().tolist()
    block_length=int(len(corrs)/2)
    #find order 
    order=df['order'].tolist()[0]
    #1 congr then incong
    #2 incongr then congr
    
    if order==1:
        congr_corr=corrs[0:block_length]
        congr_rts=rts[0:block_length]
        incon_corr=corrs[block_length:]
        incon_rts=rts[block_length:]
    else:
        congr_corr=corrs[block_length:]
        congr_rts=rts[block_length:]
        incon_corr=corrs[0:block_length]
        incon_rts=rts[0:block_length]
            
    
    
    ## -----------ANALYSE / CALCULATE
        
    #1 discard subject if too many fast responses
    if sum(np.array(congr_rts + incon_rts)<fastRT_limit)>len(congr_rts + incon_rts)*fast_prop_limit:
        print "excluding subject for " + os.path.basename(filename) + " because too many fast responses"
    else:
        #2 Eliminate scores over 10,000 ms
             
        congr_rts,congr_corr=exclude_slows(congr_rts,congr_corr,slowRT_limit)              
        incon_rts,incon_corr=exclude_slows(incon_rts,incon_corr,slowRT_limit)
        
        #3 Calculate pooled std
        #pooled_std=pooled.std(0) #n-1 std sample std
        #(Use N not N-1 because this is the whole sample). 
        #numpy.std is population std
        pooled=congr_rts + incon_rts #all RTs from both blocks, correct and incorrect
        pooled_std=np.std(pooled)
        
        #4 Calculated adjusted means, including the penalty
        congr_adjmean=adjustedmean(congr_rts,congr_corr,penalty)
        incon_adjmean=adjustedmean(incon_rts,incon_corr,penalty)
        
        #5 Calculate the IAT, so that pro-stereotype RTs are a -ve score
        IAT=(congr_adjmean-incon_adjmean)/pooled_std
        
        simpleIAT=mean(congr_rts)-mean(incon_rts)        
        
        print "IAT for " + os.path.basename(filename) + " is : {:+.3f}".format(IAT)
        print "Mean difference (uncorrected) is {:+.3f}".format(simpleIAT)+" seconds"
        